# BCN open data SQL

In [1]:
import requests
import urllib.parse
import pandas as pd
from database import database
from pyproj import Transformer, CRS
import json
from tqdm.notebook import tqdm

url_base = 'https://opendata-ajuntament.barcelona.cat/data/api/action/datastore_search_sql?sql='
# Use generic "table" when writing the SQL query for clearer visualization

def convert_sql_query_to_bcnod_url (base, query, table_id):
    q = query.replace('table', table_id)
    q = urllib.parse.quote(q)
    return str(base + q)

In [2]:
table_id = '50c9b17f-d297-4668-bad4-e1c217580747'
sql_query = """
SELECT * from "table" 
"""

api_url = convert_sql_query_to_bcnod_url (url_base, sql_query, table_id)
response = requests.get(api_url).json()
df = pd.DataFrame(response['result']['records'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171406 entries, 0 to 171405
Data columns (total 17 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   LATITUD         171406 non-null  object
 1   SECC_EST        171381 non-null  object
 2   LLEPOST         171406 non-null  object
 3   SECC_CENS       171406 non-null  object
 4   ETRS89_COORD_X  171406 non-null  object
 5   ETRS89_COORD_Y  171406 non-null  object
 6   _full_text      171406 non-null  object
 7   CODI_CARRER     171406 non-null  object
 8   DTE             171406 non-null  object
 9   ED50_COORD_X    171406 non-null  object
 10  NUMPOST         171406 non-null  object
 11  BARRI           171406 non-null  object
 12  ED50_COORD_Y    171406 non-null  object
 13  _id             171406 non-null  int64 
 14  DIST_POST       171406 non-null  object
 15  TIPUSNUM        171406 non-null  object
 16  LONGITUD        171406 non-null  object
dtypes: int64(1), object(16)
memor

# Data cleaning and formatting

In [3]:
df = df.dropna()
columns_to_delete = ['_id','SECC_EST', '_full_text', 'CODI_CARRER', 'ETRS89_COORD_X','ETRS89_COORD_Y', 'ED50_COORD_X', 'ED50_COORD_Y', 'NUMPOST', 'BARRI', 'DIST_POST', 'TIPUSNUM', 'LLEPOST']
df = df.drop(columns = columns_to_delete)

columns_to_num = ['LATITUD', 'SECC_CENS', 'DTE', 'LONGITUD']
for col in columns_to_num:
    df[col] = pd.to_numeric(df[col])
    
# Add local coordinates   
coord_transformer = Transformer.from_crs("WGS84", "EPSG:25831") # Coordinate transformer
df['EPSG_25831_X'] = coord_transformer.transform(df['LATITUD'].to_list(),df['LONGITUD'].to_list())[0]
df['EPSG_25831_Y'] = coord_transformer.transform(df['LATITUD'].to_list(),df['LONGITUD'].to_list())[1]

# Insert in our database

In [4]:
f = open('config_file.json')
config = json.load(f)
table = "censal_section"
my_db = database(config['host_name'], config['user_name'], config['password'], config['db'])

# Create table
my_db.execute_query(""" CREATE TABLE censal_section(
id INT AUTO_INCREMENT,
DISTRICT INT NOT NULL,
CENSAL_SECTION INT NOT NULL,
LATITUDE DECIMAL(8,6) NOT NULL,
LONGITUDE DECIMAL(8,6) NOT NULL,
EPSG_25831_X INT NOT NULL,
EPSG_25831_Y INT NOT NULL,
PRIMARY KEY (id)
)
""")

MySQL Server connection successful
MySQL Database connection test successful
Query successful


In [5]:
#Insert all data
col_names = ['LATITUDE', 'CENSAL_SECTION', 'DISTRICT', 'LONGITUDE', 'EPSG_25831_X', 'EPSG_25831_Y']
columns = ", ".join(col_names)

query = 'INSERT INTO censal_section ({columns}) VALUES '.format(columns = columns)
for row in tqdm(df.index):
    vals = ", ".join("'{s}'".format(s=s.replace("'", "''")) if isinstance(s, str) else str(s) for s in df.loc[row])
    query_line = "(" + vals + ") , "
    query = query + query_line
    
query = query[:-2] + ';'
#Insert all rows at once
my_db.execute_query(query)

  0%|          | 0/171381 [00:00<?, ?it/s]

Query successful
